In [ ]:
# Исследовать структуру xml-файлов, чтобы произвести парсинг всех данных. 
# В каждом файле содержится информация об одном или нескольких объектах из случайной предметной области. 
# Перечень всех характеристик объекта может меняться (у отдельного объекта могут отсутствовать некоторые характеристики). 
# Полученные данные собрать и записать в json. 
# Выполните также ряд операций с данными:
# отсортируйте значения по одному из доступных полей
# выполните фильтрацию по другому полю (запишите результат отдельно)
# для одного выбранного числового поля посчитайте статистические характеристики (сумма, мин/макс, среднее и т.д.)
# для одного текстового поля посчитайте частоту меток

In [1]:
from bs4 import BeautifulSoup
import re
import json
import math
import collections
import pandas as pd
import xml

In [2]:
def handle_file(file_name):
    items = []
    with open(file_name, encoding = 'utf-8') as file:
        text = ""
        for row in file.readlines():
            text += row
        root = BeautifulSoup(text, 'lxml')
        for clothing in root.find_all("clothing"):
            item = {}
            for i in clothing.contents:
                if i.name is None:
                    continue
                elif i.name == "price" or i.name == "reviews":
                    item[i.name] = int(i.get_text().strip())
                elif i.name == "price" or i.name == "rating":
                    item[i.name] = float(i.get_text().strip())
                elif i.name == "new":
                    item[i.name] = i.get_text().strip() == '+'
                elif i.name == "color":
                    item[i.name] = i.get_text().strip()
                elif i.name == "exclusive" or i.name == "sporty":
                    item[i.name] = i.get_text().strip() == 'yes'
                else:
                    item[i.name] = i.get_text().strip()
            items.append(item)
    return items

In [3]:
items = []
for i in range(1,101):
  file_name = f'./4/{i}.xml'
  result = handle_file(file_name)
  items += result
print(items[1:2])

c:\Users\2emir\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


[{'id': '405032984', 'name': 'PleinSport Belts K-6816', 'category': 'Belts', 'size': 'XL', 'color': 'Желтый', 'material': 'Шелк', 'price': 443825, 'rating': 2.04, 'reviews': 682655, 'new': False, 'exclusive': True}]


In [4]:
with open('result_all_63_4.json', 'w', encoding = 'utf-8') as f:
    f.write(json.dumps(items, ensure_ascii=False))

Сортировка значений по полю 'reviews' в убывающем порядке

In [5]:
items = sorted(items, key = lambda x: x['reviews'], reverse = True)
print(items[1:5])

[{'id': '304024450', 'name': 'Reebok Gloves Z-2449', 'category': 'Gloves', 'size': 'S', 'color': 'Желтый', 'material': 'Нейлон', 'price': 520336, 'rating': 2.44, 'reviews': 998429, 'exclusive': False, 'sporty': True}, {'id': '711010335', 'name': 'Asics Skirt W-1564', 'category': 'Skirt', 'size': 'S', 'color': 'Красный', 'material': 'Шелк', 'price': 700111, 'rating': 1.33, 'reviews': 998394, 'new': False, 'sporty': False}, {'id': '790489005', 'name': 'Brooks Umbrellas N-5128', 'category': 'Umbrellas', 'size': 'M', 'color': 'Синий', 'material': 'Кожа', 'price': 976104, 'rating': 3.35, 'reviews': 998048, 'sporty': True}, {'id': '274038725', 'name': 'Vans Jeans C-2658', 'category': 'Jeans', 'size': 'XL', 'color': 'Фиолетовый', 'material': 'Полиэстер', 'price': 552209, 'rating': 3.27, 'reviews': 998013}]


Выполним фильтрацию по полю ['color']:

In [6]:
filtered_items = []
for size in items:
    if size['color'] == 'Красный':
        filtered_items.append(size)
print(filtered_items[1:5])     

[{'id': '342360081', 'name': 'Vans Jacket H-1824', 'category': 'Jacket', 'size': 'M', 'color': 'Красный', 'material': 'Лен', 'price': 449367, 'rating': 1.19, 'reviews': 996488, 'new': True}, {'id': '760424235', 'name': 'Asics Shoes B-2638', 'category': 'Shoes', 'size': 'XL', 'color': 'Красный', 'material': 'Хлопок', 'price': 741451, 'rating': 4.77, 'reviews': 994230, 'new': False, 'exclusive': False, 'sporty': False}, {'id': '476501377', 'name': 'Lacoste T-shirt B-4205', 'category': 'T-shirt', 'size': 'S', 'color': 'Красный', 'material': 'Хлопок', 'price': 354585, 'rating': 1.22, 'reviews': 991085, 'new': False, 'exclusive': False}, {'id': '879421859', 'name': 'Adidas Jeans Q-3425', 'category': 'Jeans', 'size': 'S', 'color': 'Красный', 'material': 'Шелк', 'price': 681205, 'rating': 1.03, 'reviews': 991067, 'exclusive': False, 'sporty': True}]


In [7]:
with open("result_filtered_63_4.json", "w", encoding = 'utf-8') as file:
    file.write(json.dumps(filtered_items, ensure_ascii=False))

для числового поля ['rating'] посчитаем статистические характеристики (сумма, мин/макс, среднее и т.д.)

In [8]:
stats_items = []
df = pd.DataFrame(items)
pd.set_option('display.float_format', '{:.1f}'.format)
stats = df['rating'].agg(['max', 'min', 'mean', 'median', 'std']).to_dict()
stats_items.append(stats)
print(stats_items)

[{'max': 5.0, 'min': 1.0, 'mean': 3.0180615942028965, 'median': 3.03, 'std': 1.1498265828129515}]


для текстового поля ['size'] посчитайте частоту меток

In [9]:
text_items = []
category = [item['size'] for item in items]
f1 = collections.Counter(category)
text_items.append(f1)
print(text_items)

[Counter({'L': 577, 'XL': 575, 'XXL': 564, 'S': 548, 'M': 496})]
